In [35]:
# Cell 1: Imports
# Load required libraries for data handling, numeric operations, and linear programming.
import pandas as pd
import numpy as np
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus


In [39]:
# Cell 2: Load CSV Data
# Read all CSV files containing financial data, food composition, food prices, and nutrition requirements.
import pandas as pd
import numpy as np
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus

financial_df = pd.read_csv("data/financial_data.csv")
food_df = pd.read_csv("data/food_composition.csv", skipinitialspace=True)
prices_df = pd.read_csv("data/food_prices.csv", skipinitialspace=True)
nutrition_df = pd.read_csv("data/nutrition_requirements.csv")

# Clean column names by stripping whitespace
food_df.columns = food_df.columns.str.strip()
prices_df.columns = prices_df.columns.str.strip()
nutrition_df.columns = nutrition_df.columns.str.strip()

In [37]:
# Cell 4: Select Example User and Set Budget
# Choose a specific user for demonstration and define their weekly food budget.
user_id = "U017"
user_budget = 912.50  # SAR

# Define nutrient targets for this user (can be pulled dynamically from nutrition_df)
nutrients = {"calories": 6696, "protein": 158, "vitamin_d": 60, "iron": 44}


In [40]:
# Cell 5: Merge Food Prices
# Combine food composition with average prices from the food_prices dataset.
food_df = food_df.merge(
    prices_df.groupby("food_id")["price_per_kg"].mean().reset_index(),
    on="food_id"
)


In [41]:
# Cell 6: Define LP Problem and Decision Variables
# Create the optimization problem and define continuous variables for quantities and binary variables for diversity.
prob = LpProblem("FinAgent", LpMinimize)

x = {food_id: LpVariable(f"x_{food_id}", lowBound=0) for food_id in food_df["food_id"]}
y = {food_id: LpVariable(f"y_{food_id}", cat='Binary') for food_id in food_df["food_id"]}

# Set minimum and maximum grams per food item per week
min_grams = 50
max_grams = 500

# Compute cost per gram
costs = {row.food_id: row.price_per_kg/1000 for idx,row in food_df.iterrows()}


In [42]:
# Cell 7: Add Constraints
# Ensure food quantities respect min/max per item, nutrient requirements, and budget.
for food_id in food_df["food_id"]:
    prob += x[food_id] >= min_grams * y[food_id]
    prob += x[food_id] <= max_grams * y[food_id]

nutrient_cols = {
    "calories": "calories_per_100g",
    "protein": "protein_g",
    "vitamin_d": "vitamin_d_mcg",
    "iron": "iron_mg"
}

for nutrient_key, col_name in nutrient_cols.items():
    prob += lpSum([x[f] * food_df.loc[food_df.food_id==f, col_name].values[0]/100 for f in x]) >= nutrients[nutrient_key]

prob += lpSum([x[f]*costs[f] for f in x]) <= user_budget


In [43]:
# Cell 8: Optional: Diversity Objective
# Add a small epsilon to encourage selection of multiple food items while still minimizing cost.
epsilon = 0.01
prob += lpSum([x[f]*costs[f] for f in x]) - epsilon*lpSum([y[f] for f in y])


In [44]:
# Cell 9: Solve LP Problem
# Run the solver to find the optimal meal plan based on the defined constraints and objectives.
prob.solve()


1

In [45]:
# Cell 10: Collect and Display Results
# Gather selected food items, their weekly grams, and cost, and print a readable summary.
result = []
total_cost = 0
for f in x:
    val = x[f].varValue
    if val > 0:
        cost = val * costs[f]
        result.append({"food_name": food_df.loc[food_df.food_id==f,"food_name"].values[0],
                       "grams_per_week": val,
                       "cost_sar": cost})
        total_cost += cost

result_df = pd.DataFrame(result)
print(f"User: {user_id}")
print(f"Weekly food budget: {user_budget:.2f} SAR\n")
print("Nutrition targets:\n", nutrients)
print("\nStatus:", LpStatus[prob.status])
print(f"Total optimized weekly cost: {total_cost:.2f} SAR within budget {user_budget:.2f} SAR.\n")
print("Recommended Foods:")
print(result_df.sort_values("cost_sar", ascending=False).reset_index(drop=True))


User: U017
Weekly food budget: 912.50 SAR

Nutrition targets:
 {'calories': 6696, 'protein': 158, 'vitamin_d': 60, 'iron': 44}

Status: Optimal
Total optimized weekly cost: 24.73 SAR within budget 912.50 SAR.

Recommended Foods:
                 food_name  grams_per_week   cost_sar
0  Fried Mackerel (Var 8)        392.15686  10.980392
1          Boiled Peanuts        500.00000   8.000000
2         Steamed Peanuts        246.78611   3.948578
3         Sunflower Seeds         50.00000   0.900000
4   Baked Sunflower Seeds         50.00000   0.900000
